In [1]:
from pathlib import Path
paths = [str(x) for x in Path('C:/langchain2/wiki').glob('**/*.txt')]
paths

['C:\\langchain2\\wiki\\Chikungunya.txt',
 'C:\\langchain2\\wiki\\dengue.txt',
 'C:\\langchain2\\wiki\\Malaria.txt',
 'C:\\langchain2\\wiki\\Tuberculosis.txt',
 'C:\\langchain2\\wiki\\Typhoid_fever.txt',
 'C:\\langchain2\\wiki\\mobilbert_desase\\merges.txt']

In [2]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [3]:
from tokenizers import ByteLevelBPETokenizer
# initialize
tokenizer = ByteLevelBPETokenizer()
# and train
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [4]:
# import os
# os.mkdir('C:/langchain2/wiki/mobilbert_desase')

In [5]:
tokenizer.save_model('C:/langchain2/wiki/mobilbert_desase')

['C:/langchain2/wiki/mobilbert_desase\\vocab.json',
 'C:/langchain2/wiki/mobilbert_desase\\merges.txt']

In [6]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("C:/langchain2/wiki/mobilbert_desase")



c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [7]:
tokenizer("prevent",max_length=10,padding="max_length")

{'input_ids': [0, 84, 265, 3920, 2, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}

In [8]:
import torch
def mlm(tensor):
    rend=torch.rand(tensor.shape) #random rendom
    mask_arr=(rend<0.15)*(tensor >2)
    for i in range(tensor.shape[0]):
        selection=torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i,selection]=4
    return tensor


In [9]:
paths = [str(x) for x in Path('C:/langchain2/wiki').glob('**/*.txt')]
paths

['C:\\langchain2\\wiki\\Chikungunya.txt',
 'C:\\langchain2\\wiki\\dengue.txt',
 'C:\\langchain2\\wiki\\Malaria.txt',
 'C:\\langchain2\\wiki\\Tuberculosis.txt',
 'C:\\langchain2\\wiki\\Typhoid_fever.txt',
 'C:\\langchain2\\wiki\\mobilbert_desase\\merges.txt']

In [10]:
from tqdm.auto import tqdm
input_ids=[]
mask=[]
labels=[]
for path in tqdm(paths):
    with open(path,encoding="utf-8") as f:
        lines = f.read().split("\n")
    sampale = tokenizer (lines, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels.append(sampale.input_ids)
    mask.append(sampale.attention_mask)
    input_ids.append(mlm(sampale.input_ids.detach().clone()))

100%|██████████| 6/6 [00:01<00:00,  3.88it/s]


In [11]:
input_ids[:1]

[tensor([[   0,    2,    1,  ...,    1,    1,    1],
         [   0, 2718,  320,  ...,    1,    1,    1],
         [   0,  625,  521,  ...,    1,    1,    1],
         ...,
         [   0, 2718,  320,  ...,    1,    1,    1],
         [   0, 3872,  275,  ...,    1,    1,    1],
         [   0,    2,    1,  ...,    1,    1,    1]])]

In [12]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [13]:
input_ids

tensor([[   0,    2,    1,  ...,    1,    1,    1],
        [   0, 2718,  320,  ...,    1,    1,    1],
        [   0,  625,  521,  ...,    1,    1,    1],
        ...,
        [   0, 3266, 4990,  ...,    1,    1,    1],
        [   0, 5240, 4867,  ...,    1,    1,    1],
        [   0,    2,    1,  ...,    1,    1,    1]])

In [14]:
# torch.save(input_ids,"C:/langchain2/input_ids_1/input_ids.pt")
# torch.save(mask,"C:/langchain2/input_ids_1/mask.pt")
# torch.save(labels,"C:/langchain2/input_ids_1/labels.pt")

In [15]:
input_ids[0][:100]

tensor([0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])

In [16]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings['input_ids'].shape[0]
    def __getitem__(self,i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [18]:
dataset = Dataset(encodings)

In [19]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [20]:
dataloader

In [21]:
# import pickle

# # Save the DataLoader object
# with open('C:/langchain2/cdataloader.pkl', 'wb') as f:
#     pickle.dump(dataloader, f)


In [22]:
# import pickle

# # Load the saved DataLoader object
# with open('cdataloader.pkl', 'rb') as f:
#     dataloader = pickle.load(f)


In [23]:
from transformers import RobertaConfig

In [24]:
config = RobertaConfig(
    vocab_size=30_000,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=4,
    type_vocab_size=1


)

In [25]:
# from transformers import AutoModelForMaskedLM
from transformers import RobertaForMaskedLM


In [26]:
# model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

In [27]:
model=RobertaForMaskedLM(config=config)

In [28]:
device=torch.device('cuda') if torch.cuda.is_available() else print('cpu')

In [29]:
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-3): 4 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [30]:
from transformers import AdamW

In [31]:
optim = AdamW(model.parameters(), lr=1e-5)

c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
from tqdm.auto import tqdm


In [33]:
# import torch
# import torch.optim as optim
# from tqdm import tqdm
# from torch import nn

# # set device to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # define your model, dataloader, optimizer, and loss function here

# # set hyperparameters
# epochs = 4
# learning_rate = 1e-5

# # create optimizer and loss function
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss()

# # move model and loss function to GPU
# model.to(device)
# loss_fn.to(device)

# # training loop
# for epoch in range(epochs):
#     # set model to training mode
#     model.train()

#     # iterate through data
#     for batch in tqdm(dataloader):
#         # move data to GPU
#         inputs = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         # zero the gradients
#         optimizer.zero_grad()

#         # forward pass
#         outputs = model(inputs, attention_mask=attention_mask)

#         # calculate loss
#         loss = loss_fn(outputs.logits, labels)

#         # backward pass and optimization
#         loss.backward()
#         optimizer.step()

#     # print training loss for each epoch
#     print(f"Epoch {epoch + 1} training loss: {loss.item():.3f}")

        



#     #     # forward pass
#     #     outputs = model(inputs)
#     #     loss = loss_fn(outputs, targets)

#     #     # backward pass and optimization
#     #     optimizer.zero_grad()
#     #     loss.backward()
#     #     optimizer.step()

#     # # print training loss
#     # print(f"Epoch {epoch+1}: Training loss: {loss.item():.3f}")


In [34]:
batch_size = 1
epoch = 4
loop = tqdm(dataloader, leave=True)
for i, batch in enumerate(loop):
    if i % batch_size == 0:
        optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    if (i + 1) % batch_size == 0:
        optim.step()
        loop.set_description(f'Epoch:{epoch}')
        loop.set_description(f'loss:{loss.item()}')

    # Free up GPU memory after every batch
    del input_ids, attention_mask, labels, outputs
    torch.cuda.empty_cache()


  0%|          | 0/497 [00:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 938.00 MiB (GPU 0; 6.00 GiB total capacity; 4.48 GiB already allocated; 0 bytes free; 4.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# batch_size = 1
# epoch =4
# loop = tqdm(dataloader, leave=True)
# for i, batch in enumerate(loop):
#     if i % batch_size == 0:
#         optim.zero_grad()
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     labels = batch['labels'].to(device)
#     outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#     loss = outputs.loss
#     loss.backward()
#     if (i + 1) % batch_size == 0:
#         optim.step()
#         loop.set_description(f'Epoch:{epoch}')
#         loop.set_description(f'loss:{loss.item()}')


In [ ]:
# loop = tqdm(dataloader, leave=True)
# for batch in loop:
#     optim.zero_grad()
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     labels = batch['labels'].to(device)
#     outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#     loss = outputs.loss
#     loss.backward()
#     optim.step()
#     loop.set_description(f'Epoch:{epoch}')
#     loop.set_description(f'loss:{loss.item()}')

In [2]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    print("CUDA is not available. Using CPU instead.")


Using CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU
